# Pu-238 Maker

In this notebook I will experiment with the OpenMC tools needed to estimate Pu-238 production rate due to irradiation of a NpO2/Al cermet target.  

In [1]:
import openmc
import openmc.deplete # will be needed for depletion analysis eventually

%matplotlib inline
import numpy as np

## Materials

While I would expect the driver fuel pins (or other driver fuel forms) will be of relatively easy to define composition, I do not have a firm grasp on the composition of the NpO2/Al CERMET target. What follows is my best guess based on information provided in: 

Howard, Richard H. Overview of the Plutonium-238 Supply Program's CERMET Production Target. Oak Ridge National Lab.(ORNL), Oak Ridge, TN (United States), 2019.

In [2]:
NpO2 = openmc.Material();
NpO2.add_nuclide('Np237',1);
NpO2.add_element('O',2);
NpO2.set_density('g/cc',11.143);

Al = openmc.Material();
Al.add_element('Al',1);
Al.set_density('g/cc',2.7);


target = openmc.Material.mix_materials([NpO2,Al],[0.2,0.7],'vo'); 
# note: 10% is void so expect to get a UserWarning on this.


/home/sblair/miniconda3/envs/openmc-env/lib/python3.9/site-packages/openmc/material.py:1036: UserWarning: Warning: sum of fractions do not add to 1, void fraction set to 0.10000000000000009
  warnings.warn(msg)


In [3]:
print(target.get_nuclide_atom_densities());

OrderedDict([('Np237', ('Np237', 0.004988479277370189)), ('O16', ('O16', 0.009973177287448133)), ('O17', ('O17', 3.781267292246604e-06)), ('Al27', ('Al27', 0.042183829047439676))])


In [4]:
print(target.get_mass_density())

4.1186


I think this is at least reasonable.  

## Other materials

For this simple test I will simulate one target pin in the central position of a 5x5 lattice of fuel and target.  I seriously doubt that this is a *good* arrangement but it's something and will at least allow us to test the workflow.

In [6]:
# materials for fuel pin
uo2 = openmc.Material(name='UO2')
uo2.add_element('U',1.0, enrichment=3.0)
uo2.add_element('O',2.0)
uo2.set_density('g/cc',10.45)

# cladding

# based on PNNL-15870 Rev 1. 
zirc4 = openmc.Material(name='Zirc-4')
zirc4.add_element('O',0.006790); # by default add_element in atom fraction.
zirc4.add_element('Cr',0.001741);
zirc4.add_element('Fe',0.003242);
zirc4.add_element('Zr',0.977549);
zirc4.add_element('Sn',0.010677);
zirc4.set_density('g/cc',6.56);

# gap will be modeled as a void

water = openmc.Material(name='Water')
water.set_density('g/cc',0.76);
water.add_element('H',2);
water.add_element('O',1);
water.add_s_alpha_beta('c_H_in_H2O');

materials = openmc.Materials([target, uo2, zirc4, water]);


## Geometry

Here we will model individual fuel and target pins and construct a lattice within a RPP region having reflective boundary conditions.  This will thus simulate an infinite lattice.